In [2]:
import pandas as pd
import numpy as np
import scipy as sp


In [3]:
# import libraries
from datetime import datetime, timedelta
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
from lifetimes.utils import *
from lifetimes import BetaGeoFitter
from lifetimes.plotting import plot_probability_alive_matrix, plot_frequency_recency_matrix
from lifetimes.generate_data import beta_geometric_nbd_model


In [5]:
from __future__ import division

In [6]:
tx_data = pd.read_csv('deldata_5.csv', encoding = 'unicode_escape', low_memory=False)

In [7]:
#converting the type of Invoice Date Field from string to datetime.
tx_data['OrderCreatedAt'] = pd.to_datetime(tx_data['Order Created At'])

In [11]:
#creating YearMonth field for the ease of reporting and visualization
tx_data['OrderCreatedMonth'] = tx_data['OrderCreatedAt'].map(lambda date: 100*date.year+date.month)
tx_data.tail(2)


,Order_Code,Unique order count,Customer,Status,Payment Status,Channel,Shop name,Delivery Type,Location,Area,...,Category 3,Category 4,Category 5,Order Created At (Day),Order Created At,Month Year,Unnamed: 39,Unnamed: 40,OrderCreatedAt,OrderCreatedMonth
30244,DG-2-000001-118641,1,Md Shahnur Arefin - 01777770235,Delivered,Complete,Store Front,-,Home,Dhaka,Dhanmondi,...,NaN,NaN,NaN,Sat,"August 31, 2019",August 2019,NaN,NaN,2019-08-31,201908
30245,DG-2-000001-118658,1,Anmol Ahmed Khan - 01686513885,Delivered,Complete,Store Front,-,Home,Dhaka,Basaboo,...,Home & Living » Bedding » Pillows & Covers,NaN,NaN,Sat,"August 31, 2019",August 2019,NaN,NaN,2019-08-31,201908


In [12]:
#Choosing specific channels
#tx_data = tx_data.loc[tx_data['Channel']=="Store Front"]
#tx_data = tx_data.loc[tx_data['Channel']=="Agent"]


In [14]:
tx_user = pd.DataFrame(tx_data['Customer'].unique())
tx_user.columns = ['Customer']
tx_user = tx_user[tx_user.Customer != 'Rahimafrooz order - 01847453147']
tx_user = tx_user[tx_user.Customer != 'Ringku showdagar - 01917975311']
tx_user = tx_user[tx_user.Customer != 'MD ROFIQUL ALAM - 01734332149']

In [15]:
#get the max purchase date for each customer and create a dataframe with it
tx_max_purchase = tx_data.groupby('Customer')['OrderCreatedAt'].max().reset_index()
tx_max_purchase.columns = ['Customer','MaxPurchaseDate']
tx_min_purchase = tx_data.groupby('Customer')['OrderCreatedAt'].min().reset_index()
tx_min_purchase.columns = ['Customer','MinPurchaseDate']


In [16]:
tx_min_purchase.head()

,Customer,MinPurchaseDate
0,"""Msr Siddik "" - 01849498585",2019-04-17
1,. .. HOSSAN - 01976633552,2019-07-20
2,. .. HOSSAN - 01976633552,2019-05-26
3,. F RABBY - 01727999664,2019-03-19
4,. HOSSAIN - 01630909010,2019-03-18


In [17]:
tx_user = pd.merge(tx_user,tx_max_purchase,on = "Customer")

In [18]:
tx_user = pd.merge(tx_user,tx_min_purchase,on = "Customer")

In [19]:
tx_user['AliveTime'] = (tx_user['MaxPurchaseDate'] - tx_user['MinPurchaseDate']).dt.days

In [20]:
tx_frequency = tx_data.groupby('Customer')['Order_Code'].nunique().reset_index()
tx_frequency.columns = ['Customer','Frequency']

In [21]:
tx_user = pd.merge(tx_user,tx_frequency,on = "Customer")


In [22]:
tx_timeline = tx_user


In [23]:
tx_timeline['Timeline'] = (tx_timeline['MaxPurchaseDate'].max() - tx_timeline['MinPurchaseDate']).dt.days
tx_timeline = tx_timeline[['Customer','Timeline']]


In [24]:
tx_user = tx_user.drop(['Timeline'],axis=1)


In [25]:
tx_user = pd.merge(tx_user,tx_timeline,on = "Customer").reset_index()


In [26]:
tx_user['AOT'] = 'N/A'
tx_user.loc[tx_user['Frequency']>1,'AOT']= tx_user['AliveTime']/tx_user['Frequency']


In [27]:
tx_aot=tx_user.loc[tx_user['AOT']!='N/A'].reset_index()

In [28]:
tx_final = tx_aot.sort_values(by=['Frequency'],ascending=False)

In [30]:
print("Average order time for top 500 customers is:",tx_final['Frequency'].head(500).mean(),"days")

Average order time for top 500 customers is: 10.95 days
